# Import libraries

In [1]:
! apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 -y




The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 91 not upgraded.
Need to get 430 MB of archives.
After this operation, 2506 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 14s (31.6 MB/s)

(Reading database ... 108827 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.0) ...
7Progress: [  0%] [..........................................................] 87Progress: [ 14%] [########..................................................] 8update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
(Reading database ... 108805 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
7

In [2]:
! pip install -U tensorflow==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.5/497.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.4
    Uninstalling tensorflow-2.6.4:
      Successfully uninstalled tensorflow-2.6.4
ERROR: pip's dependency resol

In [3]:
import warnings
import logging
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

2.8.0
Found GPU at: /device:GPU:0


In [5]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [6]:
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0


# Load data

Solo questo dovrebbe essere diverso durante la challenge

In [7]:
dataset_dir = '/kaggle/input/homework1/training_data_final'
image_size = (96,96)

# Dataloader and Data Augmentation

In [8]:
dir(tf.keras.applications)

['DenseNet121',
 'DenseNet169',
 'DenseNet201',
 'EfficientNetB0',
 'EfficientNetB1',
 'EfficientNetB2',
 'EfficientNetB3',
 'EfficientNetB4',
 'EfficientNetB5',
 'EfficientNetB6',
 'EfficientNetB7',
 'EfficientNetV2B0',
 'EfficientNetV2B1',
 'EfficientNetV2B2',
 'EfficientNetV2B3',
 'EfficientNetV2L',
 'EfficientNetV2M',
 'EfficientNetV2S',
 'InceptionResNetV2',
 'InceptionV3',
 'MobileNet',
 'MobileNetV2',
 'MobileNetV3Large',
 'MobileNetV3Small',
 'NASNetLarge',
 'NASNetMobile',
 'ResNet101',
 'ResNet101V2',
 'ResNet152',
 'ResNet152V2',
 'ResNet50',
 'ResNet50V2',
 'VGG16',
 'VGG19',
 'Xception',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'densenet',
 'efficientnet',
 'efficientnet_v2',
 'imagenet_utils',
 'inception_resnet_v2',
 'inception_v3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3',
 'nasnet',
 'resnet',
 'resnet50',
 'resnet_v2',
 'vgg16',
 'vgg19',
 'xception']

In [9]:
from importlib import import_module

# Create the base model from the pre-trained model
IMG_SHAPE = image_size + (3,)
batch_size = 16
validation_split=0.1
model_name = 'EfficientNetV2L'
layer_name = 'efficientnetv2-l'
cat_name = 'efficientnet_v2'
base_model = getattr(tfk.applications, model_name)(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
preprocess_input = getattr(tf.keras.applications, cat_name).preprocess_input


473186304/473176280 [==============================] - 15s 0us/step


In [10]:
noaug_noval_train_data_gen = tfk.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)


aug_train_data_gen = tfk.preprocessing.image.ImageDataGenerator(
                                        rotation_range=180,
                                        height_shift_range=20,
                                        width_shift_range=20,
                                        zoom_range=0.5,
                                        shear_range = 40,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='reflect',
                                        validation_split=validation_split,
                                        zca_whitening = True,
                                        preprocessing_function=preprocess_input
                                        )

aug_validation_data_gen = tfk.preprocessing.image.ImageDataGenerator(
                                        preprocessing_function=preprocess_input,
                                        validation_split=validation_split)

train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed,
                                               subset='training')


validation_gen = aug_validation_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed,
                                               subset='validation')

train_noval_gen_all = noaug_noval_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=batch_size,     
                                               shuffle=False,
                                               seed=seed)

Found 3191 images belonging to 8 classes.
Found 351 images belonging to 8 classes.
Found 3542 images belonging to 8 classes.


In [11]:
print("Assigned labels")
print(train_gen.class_indices)
print()
print("Target classes")
print(train_gen.classes)
num_classes = train_gen.num_classes

Assigned labels
{'Species1': 0, 'Species2': 1, 'Species3': 2, 'Species4': 3, 'Species5': 4, 'Species6': 5, 'Species7': 6, 'Species8': 7}

Target classes
[0 0 0 ... 7 7 7]


# Load pre-trained model for transfer learning

We can fine tune this with different pretrained models

In [12]:
# base_model.summary()

In [13]:
def build_model(input_shape, output_classes, learning_rate=0.001, freeze=True):
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Freeze the base model
    if(freeze):
      base_model.trainable = False
    else:
      base_model.trainable = True
    # We need training=False for the BatchNormalization layer
    feature_extractor = base_model(input_layer , training=False)

    x = tfkl.GlobalAveragePooling2D()(feature_extractor)
    x = tfkl.Dropout(0.2, seed=seed)(x)
    x = tfkl.Dense(units=512, activation='swish', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.3, seed=seed)(x)
    x = tfkl.Dense(units=128, activation='swish', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='hidden_layer_2')(x)
    x = tfkl.Dropout(0.3, seed=seed)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate), metrics='accuracy')

    # Return the model
    return model

In [14]:
def build_custom_model(input_shape, output_classes, learning_rate=0.001):
    chanDim = -1
    levels = 4
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    x = tfkl.Conv2D(32, (7, 7), padding="same")(input_layer)
    x = tfkl.LeakyReLU(alpha=0.3)(x)
    x = tfkl.BatchNormalization(axis=chanDim)(x)
    x = tfkl.Add([input_layer,x])
    # CNN 
    for i in range(levels):
        first = tfkl.MaxPooling2D(pool_size=(3, 3))(x)
        x = tfkl.Conv2D(32*(i+2), (3, 3), padding="same")(first)
        x = tfkl.LeakyReLU(alpha=0.3)(x)
        x = tfkl.BatchNormalization(axis=chanDim)(x)
        x = tfkl.Add([first,x])
    x = tfkl.MaxPooling2D(pool_size=(3, 3))(x)
    
    # FC
    x = tfkl.GlobalAveragePooling2D()(x)
    x = tfkl.Dropout(0.3)(x)
    x = tfkl.Dense(units=512, activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.3)(x)
    x = tfkl.Dense(units=128, activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.3)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed), name='output_layer')(x)
    
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')
              
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),metrics="accuracy")
    return model

In [15]:
saved_as_dataset = False
if(saved_as_dataset):
    model = tfk.models.load_model(f'/kaggle/input/efficientnetb2/{model_name}_all.h5')   
else:
    model = build_model(IMG_SHAPE, num_classes, freeze=True)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-l (Functiona  (None, 3, 3, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 hidden_layer_1 (Dense)      (None, 512)               655872    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0     

In [16]:
  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

In [17]:
train_FC = True
if(train_FC):
    history = model.fit(
            x = train_gen,
            epochs = 50,
            validation_data = validation_gen,
            callbacks = [early_stopping_callback]
        ).history
    model.save(f"./models/{model_name}_FC_only.h5")

Epoch 1/50
200/200 [==============================] - 61s 178ms/step - loss: 1.4073 - accuracy: 0.4879 - val_loss: 1.1543 - val_accuracy: 0.5755
Epoch 2/50
200/200 [==============================] - 19s 93ms/step - loss: 1.1676 - accuracy: 0.5685 - val_loss: 1.0440 - val_accuracy: 0.5926
Epoch 3/50
200/200 [==============================] - 18s 88ms/step - loss: 1.0816 - accuracy: 0.6036 - val_loss: 1.0148 - val_accuracy: 0.5954
Epoch 4/50
200/200 [==============================] - 19s 92ms/step - loss: 1.0698 - accuracy: 0.6105 - val_loss: 0.8876 - val_accuracy: 0.6496
Epoch 5/50
200/200 [==============================] - 18s 91ms/step - loss: 1.0177 - accuracy: 0.6264 - val_loss: 0.8429 - val_accuracy: 0.7037
Epoch 6/50
200/200 [==============================] - 17s 86ms/step - loss: 0.9928 - accuracy: 0.6352 - val_loss: 0.8792 - val_accuracy: 0.6667
Epoch 7/50
200/200 [==============================] - 18s 92ms/step - loss: 0.9947 - accuracy: 0.6506 - val_loss: 0.8094 - val_accuracy

In [18]:
# Fine tune also the last 15 layers of the feature extraction part
train_CNN = True
if(train_CNN):
    #model = tfk.models.load_model(f"./models/{model_name}_FC_only.h5")
    leave_freeze = len(model.get_layer(layer_name).layers) - 15
    for layer in model.get_layer(layer_name).layers[:leave_freeze]:
       layer.trainable = False
    for layer in model.get_layer(layer_name).layers[leave_freeze:]:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(),metrics="accuracy")
    model.summary()
    history = model.fit(
            x = train_gen,
            epochs = 50,
            validation_data = validation_gen,
            callbacks = [early_stopping_callback]
        ).history
    model.save(f"./models/{model_name}_CNN_tuned.h5")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-l (Functiona  (None, 3, 3, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 hidden_layer_1 (Dense)      (None, 512)               655872    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0     

In [19]:
len(model.get_layer(layer_name).layers) 

956

In [20]:
# Fine tune also the last 200 layers of the feature extraction part
train_CNN = True
if(train_CNN):
    #model = tfk.models.load_model(f"./models/{model_name}_FC_only.h5")
    leave_freeze = len(model.get_layer(layer_name).layers) - 200
    for layer in model.get_layer(layer_name).layers[:leave_freeze]:
       layer.trainable = False
    for layer in model.get_layer(layer_name).layers[leave_freeze:]:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(6e-4),metrics="accuracy")
    model.summary()
    history = model.fit(
            x = train_gen,
            epochs = 50,
            validation_data = validation_gen,
            callbacks = [early_stopping_callback]
        ).history
    model.save(f"./models/{model_name}_CNN_tuned_2.h5")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-l (Functiona  (None, 3, 3, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 hidden_layer_1 (Dense)      (None, 512)               655872    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0     

In [21]:
# Fine tune all the model
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
train_all = True
if(train_all):
    model = tfk.models.load_model(f"./models/{model_name}_CNN_tuned.h5")
    for layer in model.get_layer(layer_name).layers:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4),metrics="accuracy")
    model.summary()
    history = model.fit(
            x = train_gen,
            epochs = 200,
            validation_data = validation_gen,
            callbacks = [early_stopping_callback]
        ).history
    model.save(f"./models/{model_name}_all.h5")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-l (Functiona  (None, 3, 3, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 hidden_layer_1 (Dense)      (None, 512)               655872    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0     

In [22]:
# Retrain also on validation with low learining rate
last_train = True
model = tfk.models.load_model(f"./models/{model_name}_all.h5")
if(last_train):
    for layer in model.get_layer(layer_name).layers:
       layer.trainable = True
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.SGD(learning_rate=1e-6),metrics="accuracy")
    model.summary()
    history = model.fit(
                x = validation_gen,
                epochs = 5
            ).history
    model.save(f"./models/{model_name}_final.h5")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-l (Functiona  (None, 3, 3, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 hidden_layer_1 (Dense)      (None, 512)               655872    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0     

# Create submission

In [23]:
with open("template.py", "w") as file:
  file.write(
f"""import os
import tensorflow as tf
class model:
    def __init__(self, path):
        self.model = tf.keras.models.load_model(os.path.join(path, f"SubmissionModel/model.h5"))
        self.preprocess_input = getattr(tf.keras.applications, '{cat_name}' ).preprocess_input
    def predict(self, X):
        X = self.preprocess_input(X)
        out = self.model.predict(X)
        out = tf.argmax(out, axis=-1)
        return out""")
!cat template.py

import os
import tensorflow as tf
class model:
    def __init__(self, path):
        self.model = tf.keras.models.load_model(os.path.join(path, f"SubmissionModel/model.h5"))
        self.preprocess_input = getattr(tf.keras.applications, 'efficientnet_v2' ).preprocess_input
    def predict(self, X):
        X = self.preprocess_input(X)
        out = self.model.predict(X)
        out = tf.argmax(out, axis=-1)
        return out

In [24]:
! rm submission.zip
! rm -rf ./submission
! mkdir ./submission
! mkdir ./submission/SubmissionModel
! touch ./submission/metadata
! cp ./models/$model_name\_all.h5 ./submission/SubmissionModel/model.h5
! cp template.py ./submission/model.py
! zip -r submission.zip submission/*


rm: cannot remove 'submission.zip': No such file or directory
  adding: submission/SubmissionModel/ (stored 0%)
  adding: submission/SubmissionModel/model.h5 (deflated 6%)
  adding: submission/metadata (stored 0%)
  adding: submission/model.py (deflated 46%)


In [25]:
from IPython.display import FileLink
#FileLink(r'submission.zip')

# Test submission

In [26]:
from template import model
test_model = model('./submission')

In [27]:
image = Image.open('/kaggle/input/homework1/training_data_final/Species1/00006.jpg')
frame = np.expand_dims(np.asarray(image), 0)
pred = test_model.predict(train_noval_gen_all)

In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(pred,train_noval_gen_all.labels ))
confusion_matrix(pred,train_noval_gen_all.labels )

0.8992094861660079


array([[ 59,   2,   0,   0,   0,   0,   1,   5],
       [ 33, 516,   0,   2,   1,   7,   7,  52],
       [  2,   1, 459,   0,  12,   0,   1,   1],
       [ 31,   3,  17, 499,  16,   1,   1,  16],
       [  4,   0,  39,   8, 500,   1,   0,   0],
       [  6,   1,   0,   0,   1, 211,   1,  11],
       [  2,   5,   0,   0,   0,   1, 525,   7],
       [ 49,   4,   0,   2,   1,   1,   1, 416]])